In [1]:
import warnings, json, numpy as np, pandas as pd, dash, plotly.express as px, plotly.graph_objects as go
from textwrap import dedent as d
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash

ruta = 'C://Tweetsentpol.csv'
df = pd.read_csv(ruta, header=0)
df2 = df[['Fecha', 'Polarity']]

fecha_ini = '2021-08-01'
fecha_fin = '2021-12-31'
mask = (df2['Fecha'] >= fecha_ini) & (df2['Fecha'] <= fecha_fin)
df3 = df2.loc[mask]
df3 = df3.groupby(['Fecha']).mean()
df4 = df3.reset_index()
df4['Color'] = "Fecha"

warnings.simplefilter(action='ignore', category=FutureWarning)

app = JupyterDash(__name__)
styles = {
    'pre': {
        'border': 'thin lightgrey solid',
        'overflowX': 'scroll'
    }
}

fig = px.line(df4, x="Fecha", y="Polarity", color="Color", title="No label selected")
fig.update_traces(mode="markers+lines")

app.layout = html.Div([
    dcc.Graph(
        id='figure1',
        figure=fig,
    ),
    
    html.Div(className='row', children=[
        html.Div([
            dcc.Markdown(d("""Hoverdata using figure references""")),
            html.Pre(id='hoverdata2', style=styles['pre']),
        ], className='three columns'),
    ])
])

store = []

ruta2 = 'C://Tweets.csv'
dftuits = pd.read_csv(ruta2, header=0)

In [2]:
@app.callback(
    Output('hoverdata2', 'children'),
    [Input('figure1', 'clickData')])

def display_hover_data(hoverData):
    if hoverData is not None:
        traceref = hoverData['points'][0]['curveNumber']
        pointref = hoverData['points'][0]['pointNumber']

        mask2 = (dftuits['fecha'] == fig.data[traceref]['x'][pointref])
        dftuits2 = dftuits.loc[mask2]
        lostuits = pd.Series(dftuits2['tweet'])
        lostuits2 = lostuits.astype(str).str.cat(sep='\n')

        return html.Div([
                html.Br(),html.Pre('Dia Seleccionado ' + fig.data[traceref]['x'][pointref]),html.Br(),html.Pre(lostuits2)
            ])
    else:
        return 'None selected'
    
app.run_server(mode='external', port = 7078, dev_tools_ui=True,
          dev_tools_hot_reload =True, threaded=True)

Dash app running on http://127.0.0.1:7078/
